In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup
import joblib
from tqdm import tqdm

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# brewer2mpl.get_map args: set name set type number of colors
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

colors = ['#1E90FF', '#f12c60', '#fff21e', '#1efff2', '#1eafff', 'blueviolet', 'sandybrown', 'magenta', 'slategray']

Profile construction

In [ ]:
df_res = pd.read_csv('df_res.csv')

wind_array = df_res.to_numpy()

In [ ]:
df_res

In [ ]:
# window lenght is m
m = 1000

nrm = [[] for j in range(len(wind_array) // m - 1 )]

for i in range(0, len(wind_array) // m - 1):
    nrm[i] = wind_array[i*(m):i*(m) + m]

In [ ]:
len(nrm)

201

In [ ]:
def find_nearest(a, a0):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    return idx

DPC

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

tfd = tfp.distributions

In [ ]:
def BNN_normal():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(64, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x = Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  # outputs = Dense(1)(mean_x)
  model = keras.Model(inputs=[mean_inputs], outputs=[outputs])
  return model

def BNN_gamma1():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  outputs=Dense(1)(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

def BNN_gamma2():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x=Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Gamma(concentration=1e-3+tf.abs(t[..., 0]),rate=1e-3+tf.abs(t[..., 1])))(mean_x)
  model = keras.Model(inputs=mean_inputs,outputs=outputs)
  return model

import keras.backend as K
import math as m

def negative_loglikelihood(targets,estimated_distribution):
  return -estimated_distribution.log_prob(targets)

In [12]:
len(nrm)

201

In [ ]:
for m in range(192, len(nrm)): #

  nrm0 = nrm[m]
  speed0 = nrm0[:, 0::2]
  power0 = nrm0[:, 1::2]

  omega = 0.2
  std = np.std(speed0[:,0])
  speed0_new = [[] for _ in range(14)]
  power0_new = [[] for _ in range(14)]
  speed0_new[0] = speed0[:,0]
  power0_new[0] = power0[:,0]

  for j in range(1, 14):
    for i in range(len(speed0[:,0])):
      idx = find_nearest(speed0[:,j], speed0[:,0][i])
      tmp = speed0[:,j].flat[idx]
      if abs(tmp - speed0[:,0][i]) <  omega*std:
        speed0_new[j].append(tmp)
        power0_new[j].append(power0[:,j].flat[idx])

  for k in range(14): #for k in range(14)
    speed_list = np.array(np.array(speed0)[:,k])
    speed_list = speed_list.reshape(len(speed_list), 1)
    power_list = np.array(power0)[:,k]/2000
    power_list[power_list<=0]=0.01
    model = BNN_gamma2()

    # Train
    learning_rate = 0.01
    # model.compile(optimizer='adam',loss='mse',metrics=[keras.metrics.RootMeanSquaredError()],)
    model.compile(optimizer='adam',loss=negative_loglikelihood,metrics=[keras.metrics.RootMeanSquaredError()],)
    loss_list = []
    model_list = []
    history = model.fit(speed_list, power_list, batch_size=256, epochs=995, validation_split=0)

    history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
    loss_list.append(history.history['loss'][0])
    model_list.append(model)
    history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
    loss_list.append(history.history['loss'][0])
    model_list.append(model)
    history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
    loss_list.append(history.history['loss'][0])
    model_list.append(model)
    history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
    loss_list.append(history.history['loss'][0])
    model_list.append(model)
    history = model.fit(speed_list, power_list, batch_size=64, epochs=1, validation_split=0)
    loss_list.append(history.history['loss'][0])
    model_list.append(model)
    ind = np.argmin(loss_list)
    res_model = model_list[ind]
    model_save_path = "models/Nw1000_DPC/density_%s_%s.h5"%(m, k)
    # 保存模型权重
    res_model.save_weights(model_save_path)